In [3]:
# load core libraries for EDA

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
sns.set(style="whitegrid", palette="deep")

In [4]:
# ingest data
case_dat = pd.read_csv("2024_consolidated_ai_inventory_raw_v2.csv", encoding='windows-1252')

case_dat.shape

(2133, 62)

## Data Dictionary Summary — Federal AI Use Case Inventory

The **`data_dictionary_clean.csv`** file documents the structure and meaning of variables contained in the **2024 Federal AI Use Case Inventory** dataset.  
It serves as a reference for aligning raw data fields with analytical and modeling requirements in the Deloitte–Georgetown Capstone project.

### Overview
- **Total variables:** 62  
- **Coverage:** Describes federal AI use cases across multiple agencies and sectors.  
- **Focus:** Each record represents an individual AI application reported under Section 5 of *Executive Order 13960: Promoting the Use of Trustworthy Artificial Intelligence in the Federal Government.*

### Key Variable Categories

| **Category** | **Example Fields** | **Purpose** |
|---------------|--------------------|--------------|
| **Narrative (Text)** | `description`, `intended_purpose`, `expected_benefits`, `system_outputs`, `54_key_risks` | Provide qualitative context on system function, goals, and identified risks. These fields are central for NLP/text classification. |
| **Governance & Compliance** | `52_impact_assessment`, `56_monitor_postdeploy`, `55_independent_eval`, `65_appeal_process`, `57_autonomous_impact` | Capture oversight and accountability measures—strong predictors of risk maturity. |
| **Context & Metadata** | `agency`, `bureau`, `use_case_topic_area`, `16_dev_stage`, `date_implemented`, `date_retired` | Describe organizational, functional, and temporal dimensions of each AI system. |
| **Impact Labels** | `impact_type` | The target variable indicating whether a use case is *safety-impacting*, *rights-impacting*, *both*, or *neither.* |

### Analytical Relevance
- Serves as a **feature guide** for the text classification model predicting safety or rights impact.
- Enables consistent **feature naming and cleaning** across all preprocessing scripts.
- Helps distinguish between **policy-relevant variables** (e.g., oversight processes) and **informational fields** (e.g., URLs, dates).



## Data Dictionary Summary — YAML Format

The **`data_dictionary.yaml`** file contains structured metadata describing every field in the **2024 Federal AI Use Case Inventory** dataset.  
Unlike a CSV-based dictionary, the YAML format stores **hierarchical information** — allowing nested categories (e.g., `narrative`, `governance`, `contextual`) and attributes (e.g., `type`, `description`, `priority`, `source`).

### Overview
- **Total fields:** ~62 variables  
- **Format:** YAML (YAML Ain’t Markup Language)  
- **Purpose:** Provide standardized definitions and categories for variables used in the capstone’s classification model.  
- **Usage:** Enables cleaner preprocessing, better feature alignment, and direct referencing during feature selection and documentation.

### Main Structure
Typical entries in the YAML file follow this structure:
```yaml
description:
  type: text
  category: narrative
  priority: must-have
  description: "Free-text field describing AI system purpose and scope."
52_impact_assessment:
  type: categorical
  category: governance
  priority: should-have
  description: "Indicates whether an impact assessment has been conducted."
use_case_topic_area:
  type: categorical
  category: context
  priority: must-have
  description: "Functional area of the AI use case (e.g., law enforcement, health, services)."


In [5]:
# how to safely read yaml:
import yaml

# Read YAML file
with open("data_dictionary.yaml", "r", encoding="utf-8") as f:
    dictionary = yaml.safe_load(f)

# Inspect the top-level structure
print(type(dictionary))
print(list(dictionary.keys())[:10])  # show first 10 keys


<class 'dict'>
['fields']


In [6]:
# extract the actual field dictionary
fields = dictionary['fields']

# Inspect first two entries
fields[:3]

[{'name': '2_use_case_name',
  'type': 'string',
  'description': 'Title of the AI use case.',
  'constraints': {'required': True}},
 {'name': '3_agency',
  'type': 'string',
  'description': 'Agency responsible for the AI use case.',
  'constraints': {'required': True}},
 {'name': '3_abr',
  'type': 'string',
  'description': 'Agency abbreviation.',
  'constraints': {'required': True}}]

In [7]:
# Convert YAML list to DataFrame
df_dict = pd.DataFrame(dictionary['fields'])
df_dict.head()

,name,type,description,constraints,custom_constraints,conditional_required
0,2_use_case_name,string,Title of the AI use case.,{'required': True},NaN,NaN
1,3_agency,string,Agency responsible for the AI use case.,{'required': True},NaN,NaN
2,3_abr,string,Agency abbreviation.,{'required': True},NaN,NaN
3,4_bureau,string,Specific organization(s) within the agency res...,{'required': True},NaN,NaN
4,8_topic_area,string,The topic area that most closely aligns with t...,"{'required': True, 'enum': ['Government Servic...",NaN,NaN


#### compare date set vs. dictinary to see if there are missing or extrat features in the data set:

In [8]:
# extract and save feature or column names from both files
yaml_fields = set(df_dict["name"])
csv_fields = set(case_dat.columns)

missing_in_csv = yaml_fields - csv_fields # missing in csv
extra_in_csv = csv_fields - yaml_fields   # extra in csv

print(f"Fields in YAML but missing in dataset: {len(missing_in_csv)}")
print(missing_in_csv)
print("\nFields in dataset but not in YAML: ", len(extra_in_csv))
print(extra_in_csv)


Fields in YAML but missing in dataset: 62
{'61_adverse_impact', '59_ai_notice', '19_date_acq_dev_began', '8_question_type', '47_question_type', '45_compute_request', '45_question_type', '62_disparity_mitigation', '66_no_appeal_reason', '16_dev_stage', '18_date_initiated', '38_code_access', '63_stakeholder_consult', '31_question_type', '11_purpose_benefits', '40_has_ato', '39_code_link', '23_contract_piids', '21_date_retired', '28_iqa_compliance', '50_internal_review', '54_key_risks', '3_agency', '20_date_implemented', '27_public_info', '67_opt_out', '53_real_world_testing', '34_data_docs', '31_data_catalog', '63_question_type', '37_custom_code', '25_hisp_name', '55_independent_eval', '24_hisp_support', '33_agency_data', '30_saop_review', '59_question_type', '43_question_type', '29_contains_pii', '65_appeal_process', '67_question_type', '3_abr', '51_extension_request', '57_autonomous_impact', '22_dev_method', '52_impact_assessment', '12_outputs', '56_monitor_postdeploy', '2_use_case_nam

# 🧩 Workflow Overview — Aligning Dataset Columns with YAML Schema

This section outlines the **conceptual workflow** for standardizing the dataset’s column names to the official YAML schema (short feature codes such as `52_impact_assessment`).

The focus here is **what will be executed and why**, not the actual code.

---

## **Step 0 — Inputs and Setup**
- **Files involved:**
  - `case_dat`: the main dataset (CSV), where each column is a *long, human-readable question*.
  - `df_dict`: the YAML dictionary, where each *row defines a feature* with a short **code** (`name`) and its full **question text** (`description`).

**Purpose:**  
These two files use different naming conventions for the same data — we will reconcile them so the dataset follows the YAML schema.

---

## **Step 1 — Build the Mapping (CSV → YAML Codes)**
**Goal:** Connect each CSV column header to its corresponding YAML code.

**Process:**
1. Normalize both sets of names (lowercase, remove punctuation and special characters).
2. Match CSV headers to YAML descriptions exactly.
3. If no exact match, use normalized and fuzzy string matching to find likely pairs.
4. Build a dictionary that maps each long CSV question to its short YAML code.

**Why:**  
This step ensures every feature is referenced by its stable schema code while keeping a record of its descriptive label.

---

## **Step 2 — Audit Coverage**
**Goal:** Verify that every column and YAML feature are properly matched.

**Checks performed:**
- **Duplicates:** detect if two CSV columns map to the same YAML code.  
- **Unmapped CSV columns:** identify dataset fields that have no corresponding YAML entry.  
- **Missing YAML features:** find any YAML codes not present in the dataset.

**Why:**  
Catching mismatches early ensures you don’t lose data or duplicate features when renaming.

---

## **Step 3 — Manual Fixes**
**Goal:** Resolve mismatches or ambiguous cases flagged in the audit.

**Process:**
- Review unmapped or duplicate items.
- Add a few manual corrections (e.g., confirm that *“Agency”* corresponds to `3_agency`).
- Finalize a one-to-one mapping between all dataset columns and YAML codes.

**Why:**  
Manual validation ensures the final mapping is both technically correct and semantically accurate.

---

## **Step 4 — Apply the Rename and Create a Lookup**
**Goal:** Produce a standardized dataset that uses YAML codes as feature names.

**Process:**
- Rename the dataset columns using the verified mapping.
- Create a **lookup table** linking each YAML code to its full question text for human readability.

**Why:**  
- Short YAML codes make analysis and modeling cleaner and more consistent.
- The lookup table preserves context for policy interpretation and reporting.

**Outcome:**
- `case_dat_std`: dataset with standardized YAML-coded columns.  
- `lookup`: table showing each YAML code with its full question description.

---

## **Step 5 — Save Artifacts and Ensure Reproducibility**
**Goal:** Document and preserve your alignment work.

**Process:**
- Save the mapping file (`column_mapping_csv_to_yaml_codes.csv`).
- Save both datasets:
  - Raw version (original headers).
  - Standardized version (YAML-coded headers).
- Save the lookup reference for future analyses or reports.

**Why:**  
These files serve as permanent documentation of how your dataset aligns with the official schema and make collaboration easy.

---

## ✅ **End Result**
After completing these steps, you’ll have:
1. A **YAML-aligned dataset** (`case_dat_std`) — ideal for analytics and modeling.  
2. A **mapping file** documenting all feature name relationships.  
3. A **human-readable lookup** for easy interpretation and reporting.  
4. A **reproducible, transparent workflow** ready for future dataset updates.


In [9]:
# --- SECTION 1: Build mapping from CSV headers → YAML codes (via YAML descriptions) ---

import re, difflib

# - case_dat: loaded CSV dataset
# - df_dict: DataFrame for YAML with columns at least ['name','description']

# define fucntion to normalize text (feature name)
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[_/|:–—-]+", " ", s)
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s

yaml_codes = df_dict['name'].astype(str)
yaml_descs = df_dict['description'].astype(str)
csv_cols   = pd.Series(case_dat.columns.astype(str), name='csv_name')

# bridges (YAML description -> YAML code)
desc_to_code = dict(zip(yaml_descs, yaml_codes))
norm_desc_to_code = dict(zip(yaml_descs.map(_norm), yaml_codes))

csv_to_yaml = {}
unmatched = []

# 1) exact + normalized exact
for col in csv_cols:
    if col in desc_to_code:
        csv_to_yaml[col] = desc_to_code[col]
    else:
        ncol = _norm(col)
        if ncol in norm_desc_to_code:
            csv_to_yaml[col] = norm_desc_to_code[ncol]
        else:
            unmatched.append(col)

# 2) fuzzy match for remaining columns against YAML descriptions
yaml_desc_norm_list = yaml_descs.map(_norm).tolist()
for col in unmatched:
    ncol = _norm(col)
    best = difflib.get_close_matches(ncol, yaml_desc_norm_list, n=1, cutoff=0.70)
    if best:
        idx = yaml_desc_norm_list.index(best[0])
        csv_to_yaml[col] = yaml_codes.iloc[idx]

# Preview mapping progress
mapped_pairs = pd.DataFrame(list(csv_to_yaml.items()), columns=['csv_name','yaml_code'])
print("Mapped (CSV → YAML):", len(mapped_pairs), "of", len(csv_cols))
mapped_pairs.head(10)


Mapped (CSV → YAML): 11 of 62


,csv_name,yaml_code
0,Agency Abbreviation,3_abr
1,Was the AI system involved in this use case de...,22_dev_method
2,Provide the Procurement Instrument Identifier(...,23_contract_piids
3,Which public-facing service is the AI use case...,26_public_service
4,Does this AI use case disseminate information ...,27_public_info
5,Does this AI use case involve personally ident...,29_contains_pii
6,Has the Senior Agency Official for Privacy (SA...,30_saop_review
7,Does this project include custom-developed code?,37_custom_code
8,Does the agency have access to the code associ...,38_code_access
9,Does this AI use case have an associated Autho...,40_has_ato


In [10]:
# --- SECTION 2: Audit Coverage (Duplicates, Unmapped CSV, Missing YAML) ---

# 1) Check for duplicate YAML codes
dup_yaml = mapped_pairs['yaml_code'].value_counts()
dup_yaml = dup_yaml[dup_yaml > 1]

print(f"🔁 Duplicate YAML targets: {len(dup_yaml)}")
if not dup_yaml.empty:
    display(mapped_pairs[mapped_pairs['yaml_code'].isin(dup_yaml.index)].sort_values('yaml_code'))

# 2) Identify CSV columns that are still unmapped
unmapped_csv = [c for c in case_dat.columns if c not in csv_to_yaml]
print(f"❌ Unmapped CSV columns: {len(unmapped_csv)} of {len(case_dat.columns)}")
if unmapped_csv:
    display(pd.DataFrame(unmapped_csv, columns=['csv_unmapped']))

# 3) Identify YAML codes that were not found in the dataset
yaml_codes_set = set(df_dict['name'])
mapped_codes_set = set(mapped_pairs['yaml_code'])
missing_yaml = yaml_codes_set - mapped_codes_set

print(f"⚠️ YAML codes missing from dataset: {len(missing_yaml)} of {len(yaml_codes_set)}")
if missing_yaml:
    display(pd.DataFrame(sorted(list(missing_yaml)), columns=['yaml_code_missing']))

# Quick visual summary
print("\n✅ Mapped:", len(mapped_pairs))
print("🟡 Still Unmapped:", len(unmapped_csv))
print("❌ Missing YAML Codes:", len(missing_yaml))


🔁 Duplicate YAML targets: 0
❌ Unmapped CSV columns: 51 of 62


,csv_unmapped
0,Use Case Name
1,Agency
2,Bureau
3,Use Case Topic Area
4,Other (Use Case Topic Area)
5,Is the AI use case found in the below list of ...
6,What is the intended purpose and expected bene...
7,Describe the AI system’s outputs.
8,Stage of Development
9,"Is the AI use case rights-impacting, safety-im..."


⚠️ YAML codes missing from dataset: 51 of 62


,yaml_code_missing
0,10_commercial_ai
1,11_purpose_benefits
2,12_outputs
3,16_dev_stage
4,17_impact_type
5,18_date_initiated
6,19_date_acq_dev_began
7,20_date_implemented
8,21_date_retired
9,24_hisp_support



✅ Mapped: 11
🟡 Still Unmapped: 51
❌ Missing YAML Codes: 51


In [11]:
# --- SECTION 3: Manual Fixes & Re-Audit ---

# 1) Extend mapping with the most common known pairs.
manual_additions = {
    "Use Case Name": "2_use_case_name",
    "Agency": "3_agency",
    "Bureau": "4_bureau",
    "Use Case Topic Area": "8_topic_area",
    "Stage of Development": "16_dev_stage",
    "Is the AI use case rights-impacting, safety-impacting, both, or neither?": "17_impact_type",
    "Date Initiated": "18_date_initiated",
    "Date when Acquisition and/or Development began": "19_date_acq_dev_began",
    "Date Implemented": "20_date_implemented",
    "Date Retired": "21_date_retired",
    "Which HISP is the AI use case supporting?": "25_hisp_name",
    "Is this AI use case supporting a High-Impact Service Provider (HISP) public-facing service?": "24_hisp_support",
    "How is the agency ensuring compliance with Information Quality Act guidelines, if applicable?": "28_iqa_compliance",
    "Do you have access to an enterprise data catalog or agency-wide data repository that enables you to identify whether or not the necessary datasets exist and are ready to develop your use case?": "31_data_catalog",
    "Describe any agency-owned data used to train, fine-tune, and/or evaluate performance of the model(s) used in this use case.": "33_agency_data",
    "Is there available documentation for the model training and evaluation data that demonstrates the degree to which it is appropriate to be used in analysis or for making predictions?": "34_data_docs",
    "Which, if any, demographic variables does the AI use case explicitly use as model features?": "35_demo_features",
    "If the code is open-source, provide the link for the publicly available source code.": "39_code_link",
    "System Name": "41_system_name",
    "How long have you waited for the necessary developer tools to implement the AI use case? ": "42_dev_tools_wait",
    "For this AI use case, is the required IT infrastructure provisioned via a centralized intake form or process inside the agency?": "43_infra_provisioned",
    "Do you have a process in place to request access to computing resources for model training and development of the AI involved in this use case?": "45_compute_request",
    "Has your agency requested an extension to implement the minimum risk management practices for this AI use case?": "51_extension_request",
    "Has an AI impact assessment been conducted for this AI use case?": "52_impact_assessment",
    "Has the AI use case been tested in operational or real-world environments to understand the performance and impact it may have on affected individuals or communities?": "53_real_world_testing",
    "What are the key risks from using the AI for this particular use case and how were they identified?": "54_key_risks",
    "Has an independent evaluation of the AI use case been conducted?": "55_independent_eval",
    "Is there a process to monitor performance of the AI system’s functionality and changes to its impact on rights or safety as part of the post-deployment plan for the AI use case?": "56_monitor_postdeploy",
    "For this particular use case, can the AI carry out a decision or action without direct human involvement that could result in a significant impact on rights or safety?": "57_autonomous_impact",
    "How is the agency providing reasonable and timely notice regarding the use of AI when people interact with an AI-enabled service as a result of this AI use case?": "59_ai_notice",
    "What steps has the agency taken to detect and mitigate significant disparities in the model’s performance across demographic groups for this AI use case?": "62_disparity_mitigation",
    "What steps has the agency taken to consult and incorporate feedback from groups affected by this AI use case?": "63_stakeholder_consult",
    "Is there an established fallback and escalation process for this AI use case in the event that an impacted individual or group would like to appeal or contest the AI system’s outcome?": "65_appeal_process",
    "Where practicable and consistent with applicable law and governmentwide policy, is there an established mechanism for individuals to opt-out from the AI functionality in favor of a human alternative?": "67_opt_out",
    "What is the intended purpose and expected benefits of the AI?": "11_purpose_benefits",
    "Describe the AI system’s outputs.": "12_outputs"
}

# 2) Merge with the existing mapping
csv_to_yaml.update(manual_additions)

# 3) Rebuild mapping table and re-check coverage
mapped_pairs = pd.DataFrame(list(csv_to_yaml.items()), columns=['csv_name','yaml_code'])
yaml_codes_set = set(df_dict['name'])
mapped_codes_set = set(mapped_pairs['yaml_code'])

unmapped_csv = [c for c in case_dat.columns if c not in csv_to_yaml]
missing_yaml = yaml_codes_set - mapped_codes_set

print(f"✅ Total mapped now: {len(mapped_pairs)} of {len(case_dat.columns)}")
print(f"🟡 Still unmapped CSV columns: {len(unmapped_csv)}")
print(f"❌ YAML codes still missing: {len(missing_yaml)}")

# show any remaining for review
if unmapped_csv:
    display(pd.DataFrame(unmapped_csv, columns=['csv_unmapped']))
if missing_yaml:
    display(pd.DataFrame(sorted(list(missing_yaml)), columns=['yaml_code_missing']))


✅ Total mapped now: 47 of 62
🟡 Still unmapped CSV columns: 15
❌ YAML codes still missing: 15


,csv_unmapped
0,Other (Use Case Topic Area)
1,Is the AI use case found in the below list of ...
2,"If Other, please explain."
3,"If Other, please explain..1"
4,"If Other, please explain..2"
5,"If Other, please explain..3"
6,"If Other, please explain..4"
7,"How are existing data science tools, libraries..."
8,"Has information regarding the AI use case, inc..."
9,"If Other, please explain..5"


,yaml_code_missing
0,10_commercial_ai
1,31_question_type
2,35_question_type
3,43_question_type
4,47_question_type
5,47_timely_resources
6,49_existing_reuse
7,50_internal_review
8,57_question_type
9,59_question_type


In [12]:
# --- SECTION 3B: Final manual fixes + auto-assign 'If Other...' columns ---

# 1) Straightforward remaining pairs
more_manual = {
    "Other (Use Case Topic Area)": "8_question_type",
    "Is the AI use case found in the below list of general commercial AI products and services?": "10_commercial_ai",
    "How are existing data science tools, libraries, data products, and internally-developed AI infrastructure being re-used for the current AI use case?": "49_existing_reuse",
    "Has information regarding the AI use case, including performance metrics and intended use of the model, been made available for review and feedback within the agency?": "50_internal_review",
    "Is the AI used to significantly influence or inform decisions or actions that could have an adverse or negative impact on specific individuals or groups?": "61_adverse_impact",
    "If No, please explain.": "66_no_appeal_reason",
}
csv_to_yaml.update(more_manual)

# 2) Auto-map the remaining "If Other..." columns in column order to question_type codes
#    (excluding 8_question_type which we've already mapped via "Other (Use Case Topic Area)")
if_other_cols = [c for c in case_dat.columns if c.startswith("If Other, please explain")]
if_other_cols_sorted = [c for c in case_dat.columns if c in if_other_cols]  # keep original CSV order

question_type_codes_ordered = [
    "31_question_type",
    "35_question_type",
    "43_question_type",
    "47_question_type",
    "57_question_type",
    "59_question_type",
    "63_question_type",
    "67_question_type",
]

# Assign in order (excess "If Other" columns, if any, will be left for manual review)
for c, code in zip(if_other_cols_sorted, question_type_codes_ordered):
    csv_to_yaml[c] = code

# 3) Rebuild mapping + re-audit
mapped_pairs = pd.DataFrame(list(csv_to_yaml.items()), columns=['csv_name','yaml_code'])
yaml_codes_set = set(df_dict['name'])
mapped_codes_set = set(mapped_pairs['yaml_code'])

unmapped_csv = [c for c in case_dat.columns if c not in csv_to_yaml]
missing_yaml = yaml_codes_set - mapped_codes_set

print(f"✅ Total mapped now: {len(mapped_pairs)} of {len(case_dat.columns)}")
print(f"🟡 Still unmapped CSV columns: {len(unmapped_csv)}")
print(f"❌ YAML codes still missing: {len(missing_yaml)}")

# Show any remaining items for a quick spot-check
if unmapped_csv:
    display(pd.DataFrame(unmapped_csv, columns=['csv_unmapped']))
if missing_yaml:
    display(pd.DataFrame(sorted(list(missing_yaml)), columns=['yaml_code_missing']))


✅ Total mapped now: 61 of 62
🟡 Still unmapped CSV columns: 1
❌ YAML codes still missing: 1


,csv_unmapped
0,"If Other, please explain..8"


,yaml_code_missing
0,47_timely_resources


In [13]:
# --- SECTION 3C: Final tiny fixes ---

# 1) Try to map the 'timely resources' question if it exists in your CSV
timely_col = "Has communication regarding the provisioning of your requested resources been timely?"
if timely_col in case_dat.columns:
    csv_to_yaml[timely_col] = "47_timely_resources"

# 2) Handle the last 'If Other, please explain..8'
# If you know the exact parent question, set that *_question_type here instead.
other8 = "If Other, please explain..8"
if other8 in case_dat.columns:
    # Provisional best-guess: tie to the 47 set (dev tools/resources) unless you confirm a different parent.
    csv_to_yaml[other8] = "47_question_type"

# Recompute audit quickly
mapped_pairs = pd.DataFrame(list(csv_to_yaml.items()), columns=['csv_name','yaml_code'])
yaml_codes_set = set(df_dict['name'])
mapped_codes_set = set(mapped_pairs['yaml_code'])

unmapped_csv = [c for c in case_dat.columns if c not in csv_to_yaml]
missing_yaml = yaml_codes_set - mapped_codes_set

print(f"✅ Total mapped now: {len(mapped_pairs)} of {len(case_dat.columns)}")
print(f"🟡 Still unmapped CSV columns: {len(unmapped_csv)}")
print(f"❌ YAML codes still missing: {len(missing_yaml)}")
if unmapped_csv:
    display(pd.DataFrame(unmapped_csv, columns=['csv_unmapped']))
if missing_yaml:
    display(pd.DataFrame(sorted(list(missing_yaml)), columns=['yaml_code_missing']))


✅ Total mapped now: 62 of 62
🟡 Still unmapped CSV columns: 0
❌ YAML codes still missing: 1


,yaml_code_missing
0,45_question_type


In [14]:
# --- SECTION 4: Apply rename + create standardized dataset + lookup ---

# 1) Standardize dataset columns to YAML codes using the mapping you built (csv_to_yaml)
case_dat_std = case_dat.rename(columns=csv_to_yaml).copy()

# 2) Sanity check: how many standardized (YAML) columns do we have?
yaml_set = set(df_dict['name'])
standardized_cols = set(case_dat_std.columns) & yaml_set
print(f"🧩 Standardized (YAML) columns present: {len(standardized_cols)} / {len(yaml_set)}")

# 3) Human-readable lookup table (YAML code ↔ full question text)
lookup = (
    df_dict[['name','description']]
    .drop_duplicates()
    .rename(columns={'name':'yaml_code','description':'question_text'})
)

present_lookup = (
    pd.DataFrame(sorted(list(standardized_cols)), columns=['yaml_code'])
    .merge(lookup, on='yaml_code', how='left')
)

print("✅ Created: case_dat_std (YAML-coded) and present_lookup (for reporting).")
present_lookup.head(10)


🧩 Standardized (YAML) columns present: 61 / 62
✅ Created: case_dat_std (YAML-coded) and present_lookup (for reporting).


,yaml_code,question_text
0,10_commercial_ai,Whether the use case is implemented solely wit...
1,11_purpose_benefits,"Description of the AI’s intended purpose, incl..."
2,12_outputs,"Description of what the AI system outputs, whe..."
3,16_dev_stage,The current stage of System Development Life C...
4,17_impact_type,Whether the AI use case is rights-impacting or...
5,18_date_initiated,Date when the AI use case’s purpose and high-l...
6,19_date_acq_dev_began,Date when the acquisition and/or development o...
7,20_date_implemented,Date when the AI use case was fully implemente...
8,21_date_retired,Date when the AI use case was retired or began...
9,22_dev_method,Whether the AI system involved in the use case...


# 🔁 Iterative Mapping Workflow — Key Python Packages & Functions

Below are the **major packages** and the **core functions/methods** used in each phase of the schema-alignment workflow, so your notebook stays well-documented and reproducible.

---

## 1) Automatic Matching (normalize + exact/normalized/fuzzy)
**Packages**
- `pandas` (`pd`)
- `re` (regex)
- `difflib`

**Key functions/methods**
- Text normalize: `re.sub()`, `str.lower()`, `str.strip()`
- Fuzzy match: `difflib.get_close_matches()`
- Pandas structures: `pd.Series()`, `pd.DataFrame()`
- Mapping prep: `zip()`, `dict()`, `.map()`, `.astype(str)`

**Why**  
Create initial **CSV header → YAML code** pairs quickly using description-based matching.

---

## 2) Audit Coverage (duplicates, unmapped, missing)
**Packages**
- `pandas`
- Python built-ins (`set`, `len`, `sorted`)

**Key functions/methods**
- Duplicates: `value_counts()`
- Unmapped lists: list comprehensions, membership tests (`in`), set ops (`set(a) - set(b)`)
- Reporting: `pd.DataFrame()`, `.sort_values()`

**Why**  
Identify where automation failed (gaps/overlaps) before manual fixes.

---

## 3) Manual Fixes (Round 1)
**Packages**
- `pandas`

**Key functions/methods**
- Extend mapping: Python `dict.update()`
- Recompute pairs: `pd.DataFrame(list(mapping.items()))`
- Quick verify: `len()`, `set()` comparisons

**Why**  
Add **high-confidence one-to-one** mappings (e.g., `Agency` → `3_agency`) that fuzzy logic might miss.

---

## 4) Re-Audit & Refine
**Packages**
- `pandas`
- Python built-ins (`set`, `len`)

**Key functions/methods**
- Repeat Phase 2 checks: `value_counts()`, set differences
- Sort & inspect: `.sort_values()`, `.head()`

**Why**  
Confirm progress (e.g., 11 → 47 → 61 mapped) and surface remaining edge cases.

---

## 5) Final Cleanup (stragglers, “If Other…” explainers)
**Packages**
- `pandas`

**Key functions/methods**
- Ordered assignment: iterate over column lists and assign codes
- Final mapping merge: `dict.update()`
- Last audit pass: `set()` diffs, `pd.DataFrame()` views

**Why**  
Resolve remaining free-text explainers and niche fields to achieve near-complete coverage.

---

## 6) Standardize & Verify (rename + lookup)
**Packages**
- `pandas`

**Key functions/methods**
- Apply mapping: `DataFrame.rename(columns=...)`
- Sanity check: `set(df.columns) & set(yaml_codes)`, `len()`
- Build lookup: `DataFrame.merge()`, `.drop_duplicates()`, `.rename()`, `sorted()`
- (Optional) Persist: `DataFrame.to_csv()`

**Why**  
Produce modeling-ready **`case_dat_std`** with YAML codes, and a **`present_lookup`** table for human-readable reporting.

---

### Optional: Early EDA (after standardization)
**Packages**
- `pandas`, `numpy`, `matplotlib.pyplot`

**Key functions/methods**
- Shape & preview: `.shape`, `.head()`
- Label check: `value_counts(dropna=False)`
- Required fields: boolean masks on metadata, `.isna().mean()`
- Text diagnostics: `.astype(str).str.len()`, simple boxplot via `matplotlib`

---

## Quick Checklist
- [ ] Normalize text (`re`, string methods)  
- [ ] Build initial mapping (`difflib.get_close_matches`)  
- [ ] Audit coverage (duplicates/unmapped/missing)  
- [ ] Add manual mappings (`dict.update`)  
- [ ] Re-audit and finalize  
- [ ] `rename()` to YAML codes + create `lookup`  
- [ ] (Optional) `to_csv()` artifacts  
- [ ] Run early EDA on `case_dat_std`

#### Export standardized dataset and reference files

In [15]:
# --- FINAL EXPORT CELL (Excel-friendly version) ---

# 1) Save standardized dataset (YAML-coded columns)
# Save both CSV (with UTF-8-BOM) and Excel formats for safety
case_dat_std.to_csv("case_dat_standardized.yamlcols.csv", index=False, encoding="utf-8-sig")
case_dat_std.to_excel("case_dat_standardized.yamlcols.xlsx", index=False)
print("✅ Saved standardized dataset → CSV (utf-8-sig) + XLSX")

# 2) Save the mapping table (CSV header → YAML code)
mapped_pairs.to_csv("column_mapping_csv_to_yaml_codes.csv", index=False, encoding="utf-8-sig")
mapped_pairs.to_excel("column_mapping_csv_to_yaml_codes.xlsx", index=False)
print("✅ Saved column mapping → CSV + XLSX")

# 3) Save the human-readable lookup (YAML code ↔ question text)
present_lookup.to_csv("current_columns_with_questions.csv", index=False, encoding="utf-8-sig")
present_lookup.to_excel("current_columns_with_questions.xlsx", index=False)
print("✅ Saved lookup reference → CSV + XLSX (Excel-friendly)")


✅ Saved standardized dataset → CSV (utf-8-sig) + XLSX
✅ Saved column mapping → CSV + XLSX
✅ Saved lookup reference → CSV + XLSX (Excel-friendly)
